# Load Classification Model

In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
df = pd.read_csv('/content/drive/MyDrive/preprocessed_dataset_adjustdikit.csv')
df.head()

,label,tweet,refined_tweet
0,anger,soal jalan jatibarupolisi bisa gertak gubernur...,soal jalan jatibarupolisi bisa gertak gubernur...
1,anger,sesama cewek lho kayaknya harusnya lebih rasai...,sesama cewek lho kayaknya harusnya rasain haru...
2,happy,kepingin gudeg mbarek bu hj amad foto google s...,kepingin gudeg mbarek bu hj amad foto google s...
3,anger,jalan jatibarubagian wilayah tn abangpengatura...,jalan jatibarubagian wilayah tn abangpengatura...
4,happy,sharing pengalaman aja kemarin jam batalin tik...,sharing pengalaman aja kemarin jam batalin tik...


In [4]:
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(df['refined_tweet'])

label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])

print(label_encoder.classes_)

['anger' 'fear' 'happy' 'love' 'sadness']


In [5]:
from tensorflow.keras.models import load_model

model_path = '/content/drive/MyDrive/Amel_Model/my_model.h5'

model = load_model(model_path)
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 100, 50)             │         500,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 100, 50)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ (None, 100, 128)            │          44,544 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_1 (Bidirectional)      │ (None, 100, 128)            │          74,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_2 (Bidirectional)      │ (None, 100, 128)            │          74,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 100, 128)            │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_3 (Bidirectional)      │ (None, 128)                 │          74,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 6)                   │             774 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 769,320 (2.93 MB)

 Trainable params: 769,062 (2.93 MB)

 Non-trainable params: 256 (1.00 KB)

 Optimizer params: 2 (12.00 B)

In [6]:
max_len = 100

# Test Classification Model

In [7]:
def predict_mood(text):
    seq = tokenizer.texts_to_sequences([text])
    padded = pad_sequences(seq, maxlen=max_len, padding='post', truncating='post')
    prediction = model.predict(padded)
    mood = label_encoder.inverse_transform([np.argmax(prediction)])[0]
    return mood

example_text = "saya cinta dia"
predicted_mood = predict_mood(example_text)
print(f"Predicted Mood: {predicted_mood}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Predicted Mood: love


# Sistem Rekomendasi Aktivitas




*   List rekomendasi disimpan di file json, agar memudahkan proses editing atau
adjustment kalau diperlukan



In [8]:
json_file_path = '/content/drive/My Drive/recommendations/RecommendationTimeBased.json'



*   Library yang digunakan :



In [9]:
import json
from datetime import datetime
import random



*   Mood Mapping



In [10]:
mood_to_stress_mapping = {
    'anger': 'high',
    'fear': 'high',
    'happy': 'low',
    'love': 'low',
    'sadness': 'medium'
}

## Setting Keyword Intensity
[Note]: ini setting keyword untuk intensity, karena model kita cuma generate single detection buat moodnya, jadi pake keyword setting buat akalin keterbatasan biar kita punya output stress level sesuai rencana awal. kalo mungkin nanti mau di demonstrasi, bisa perhatiin keyword2 ini buat set inputnya.



In [11]:
def get_intensity_from_text(text):

    high_intensity_keywords = ["terlalu", "parah", "benci", "sangat", "banget", "kesal", "muak"]
    medium_intensity_keywords = ["sedikit", "lumayan", "agak", "cuma", "mungkin", "kurang"]
    low_intensity_keywords = ["tenang", "sedang", "biasa", "ringan", "normal","senang","cinta","suka","sayang"]

    negations = ["tidak", "gak", "nggak", "bukan"]

    if any(word in text for word in high_intensity_keywords):
        return 'high'
    elif any(word in text for word in medium_intensity_keywords):
        return 'medium'
    elif any(word in text for word in low_intensity_keywords):
        return 'low'

    if any(word in text for word in negations):
        return 'low'

In [12]:
def predict_mood(text):
    seq = tokenizer.texts_to_sequences([text])
    padded = pad_sequences(seq, maxlen=max_len, padding='post', truncating='post')
    prediction = model.predict(padded)
    mood = label_encoder.inverse_transform([np.argmax(prediction)])[0]
    return mood



*   Load json file



In [13]:
def load_recommendations():
    try:
        with open('/content/drive/My Drive/RecommendationTimeBased.json', 'r') as file:
            recommendations = json.load(file)
        return recommendations
    except FileNotFoundError:
        print("RecommendationTimeBased.json file not found!")
        return {}

recommendations_data = load_recommendations()

## Format Display
[Note]  kalau mau ubah display output, atur dari sini aja. ini distructure berdasarkan design UI MD

In [14]:
def display_recommendation(recommendation):
    current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

    print(f"Time of Recommendation: {current_time}")
    print(f"Predicted Mood: {recommendation['Predicted Mood']}")

    print(f"Stress Level: {recommendation['Stress Level']}")

    print(f"\nSympathy Message: {recommendation['Sympathy Message']}\n")
    print("\nThoughtful Suggestions:")
    for suggestion in recommendation['Thoughtful Suggestions']:
        print(f" - {suggestion}")

    print("\nTips for You:")
    for tip in recommendation['Tips']:
        print(f" - {tip}")

## Define fungsi mood & stress level

In [15]:
def recommend_activity(mood, intensity):

#[Mood Intensity]
    stress_level = mood_to_stress_mapping.get(mood, 'low')

    if intensity == 'high':
        stress_level = 'high'
    elif intensity == 'medium' and stress_level == 'low':
        stress_level = 'medium'

#[Logic Time Based]
    current_hour = datetime.now().hour
    print(f"Current hour: {current_hour}")
    if current_hour < 11:
        time_of_day = 'morning'
    elif 11 <= current_hour < 15:
        time_of_day = 'afternoon'
    elif 15 <= current_hour < 21:
        time_of_day = 'evening'
    else:
        time_of_day = 'night'

    time_based_recommendations = recommendations_data.get(stress_level, {}).get(time_of_day, [])

    # Fetch [thoughtful suggestions]
    thoughtful_suggestions = []
    if time_based_recommendations:
        thoughtful_suggestions_data = time_based_recommendations[0].get('thoughtful_suggestions', [])
        if isinstance(thoughtful_suggestions_data, list):
            thoughtful_suggestions = random.sample(
                thoughtful_suggestions_data,
                k=min(3, len(thoughtful_suggestions_data))
            )

    # Fetch [tips]
    tips = []
    if time_based_recommendations:
        tips_data = time_based_recommendations[0].get('tips_untukmu_saat_ini', [])
        if isinstance(tips_data, list) and tips_data:
            tips = random.sample(
                tips_data,
                k=min(3, len(tips_data))
            )
        else:
            print("No tips found for this stress level:", stress_level)

    # Fetch [sympathy]
    if time_based_recommendations:
        sympathy_messages = time_based_recommendations[0].get('sympathy', [])
        if isinstance(sympathy_messages, list) and sympathy_messages:
            sympathy_message = random.choice(sympathy_messages)

    return stress_level, thoughtful_suggestions, tips, sympathy_message

## Combined Prediksi Klasifikasi & Rekomendasi

In [16]:
def get_recommendation_from_sentence(text):
    predicted_mood = predict_mood(text)

    intensity = get_intensity_from_text(text)

    stress_level, thoughtful_suggestions, tips, sympathy_message = recommend_activity(predicted_mood, intensity)

    return {
        "Sympathy Message": sympathy_message,
        "Predicted Mood": predicted_mood,
        "Stress Level": stress_level,
        "Thoughtful Suggestions": thoughtful_suggestions,
        "Tips": tips,
    }

## Test

In [17]:
example_text = "hari ini saya sedih sekali"
recommendation = get_recommendation_from_sentence(example_text)

display_recommendation(recommendation)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 249ms/step
Current hour: 20
Time of Recommendation: 2024-11-26 20:28:55
Predicted Mood: sadness
Stress Level: medium

Sympathy Message: Seharian penuh, ya? Mungkin kamu udah merasa agak capek dan ada sedikit beban, tapi kamu udah ngerjain semuanya dengan baik. Itu patut dihargai banget, terutama setelah hari yang panjang kayak gini.


Thoughtful Suggestions:
 - Pikirin hal-hal yang udah kamu capai hari ini. Meskipun hari ini terasa berat, apapun yang udah kamu capai itu penting, dan kamu layak dihargai.
 - Kadang ngobrol sama teman atau keluarga bisa sangat membantu, terutama setelah hari yang penuh tantangan. Mungkin bisa ngebuka pikiran dan ngeringanin beban yang kamu rasain.
 - Coba deh tulis di jurnal tentang apa yang udah kamu capai hari ini. Walaupun sore ini mungkin terasa lebih berat, menulis bisa bantu kamu lihat kemajuan dan bikin kamu merasa lebih positif.

Tips for You:
 - Karena sekarang udah sore, coba deh luangkan waktu untuk diri sendiri dan 